In [41]:
# Libraries 

import os
import glob
import shutil
import json
import keras
import itertools
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import GlobalAveragePooling2D, Flatten, Dense, Dropout, BatchNormalization, DepthwiseConv2D, Conv2D, Input
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


# Working Directories

work_dir = '/kaggle/input/cassava-leaf-disease-classification/'
train_path = '/kaggle/input/cassava-leaf-disease-classification/train_images'


In [42]:
# Visualizing Class Distributions 

df = pd.read_csv(work_dir + 'train.csv')

df.head()

print(df.shape)


x = ['CMD', 'Healthy', 'CGM', 'CBSD','CBB']


y = df['label'].value_counts()

print(y)

plt.bar(x, y)

plt.ylabel('Frequency')
plt.xlabel('Labels')

plt.title('Distribution of Image Classes: TRAIN')


In [43]:
# Reading the labels from the json file 

f = open(work_dir + 'label_num_to_disease_map.json')

labels = json.load(f) # Stored as dict by default 


labels = {int(k):v for k,v in labels.items()} # Convert keys from strings to ints 


# Defining the working dataset

df['class_name'] = df.label.map(labels)

In [44]:
# Attaching labels to the class sets and defining them as variables

mask = df['label'] == 0
CBB = df[mask]

mask = df['label'] == 1
CBSD = df[mask]

mask = df['label'] == 2
CGM = df[mask]

mask = df['label'] == 3
CMD = df[mask]

mask = df['label'] == 4
Healthy = df[mask]


In [45]:
# Sampling images from each class
# The goal of these sample sizes is to help reduce class imbalances 

# Since this is a real dataset, we can assume that the class imbalances indicate a real-world correlation 
# with the frequencies of each disease, so I am choosing to maintain a (more muted) class imbalance in the final dataset

CBB = CBB.sample(frac=1)
CBSD = CBSD.sample(frac=1)
CGM = CGM.sample(frac=1)
CMD = CMD.sample(frac=0.9)
Healthy = Healthy.sample(frac=1)


In [46]:
##### PREPROCESSING #####


# Combining the sets of images from each class 

class_sets = [CBB, CBSD, CGM, CMD, Healthy]
f_df = pd.concat(class_sets)

print(len(f_df))

# Splits -> train: 0.8, validation: 0.2

from sklearn.model_selection import train_test_split

train, validation = train_test_split(f_df, test_size = 0.2, random_state = 42, shuffle=True, stratify = f_df['class_name'])

# Convert the images into image data using keras.preprocessing.image.ImageDataGenerator

from keras.preprocessing.image import ImageDataGenerator

# Variables for size, num_classes and batch_size

IMG_SIZE = 300
n = 5
BATCH_SIZE = 32

# Preprocessing for the train and validation datasets -> Note, ImageDataGenerator performs input normalization with the rescale arg

datagen = ImageDataGenerator(
                    width_shift_range = 0.2,
                    height_shift_range = 0.2,
                    shear_range = 0.2,
                    zoom_range = 0.2,
                    horizontal_flip = True,
                    vertical_flip = True,
                    fill_mode = 'nearest',
                    rescale = 1.0/255.0)


train_set = datagen.flow_from_dataframe(train,
                         directory = train_path,
                         seed=42,
                         x_col = 'image_id',
                         y_col = 'class_name',
                         target_size = (IMG_SIZE, IMG_SIZE),
                         class_mode = 'categorical',
                         interpolation = 'nearest',
                         shuffle = True,
                         batch_size = BATCH_SIZE)

validation_set = datagen.flow_from_dataframe(validation,
                         directory = train_path,
                         seed=42,
                         x_col = 'image_id',
                         y_col = 'class_name',
                         target_size = (IMG_SIZE, IMG_SIZE),
                         class_mode = 'categorical',
                         interpolation = 'nearest',
                         shuffle = True,
                         batch_size = BATCH_SIZE)


In [47]:
##### THE MODEL ######


## INPUT BLOCK ##

def create_model2():
    
    model = Sequential()
    model.add(Conv2D(3, 3, kernel_initializer="glorot_uniform", padding="same", input_shape = (256,256,3)))
    model.add(BatchNormalization())
    model.add(keras.layers.Activation(keras.activations.relu))
    model.add(Dropout(0.2))
    
    model.add(DepthwiseConv2D(3, kernel_initializer="glorot_uniform", padding="valid"))
    model.add(BatchNormalization())
    model.add(keras.layers.Activation(keras.activations.relu))
    model.add(Dropout(0.2))
    
    
    model.add(DepthwiseConv2D(3, kernel_initializer="glorot_uniform", padding="same"))
    model.add(BatchNormalization())
    model.add(keras.layers.Activation(keras.activations.relu))
    model.add(Dropout(0.2))
    
    model.add(DepthwiseConv2D(3, kernel_initializer="glorot_uniform", padding="valid"))
    model.add(BatchNormalization())
    model.add(keras.layers.Activation(keras.activations.relu))
    model.add(Dropout(0.2))
    
    model.add(Conv2D(3, 3, kernel_initializer="glorot_uniform", padding="valid"))
    model.add(BatchNormalization())
    model.add(keras.layers.Activation(keras.activations.relu))
    model.add(Dropout(0.2))
    
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(n, activation = 'softmax'))

    return model




    
def create_model():
    
    model = Sequential()
    model.add(EfficientNetB3(input_shape = (300, 300, 3), include_top = False, weights = None))
    model.add(GlobalAveragePooling2D())
    model.add(Flatten())
    model.add(Dense(256, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(32, activation = 'relu', bias_regularizer=tf.keras.regularizers.L1L2(l1=0.01, l2=0.001)))
    model.add(Dropout(0.2))
    model.add(Dense(n, activation = 'softmax'))
    
    return model


In [48]:
test = create_model()
test.summary()



EPOCHS = 25

STEP_SIZE_TRAIN = train_set.n//train_set.batch_size
STEP_SIZE_VALID = validation_set.n//validation_set.batch_size

print(STEP_SIZE_TRAIN, STEP_SIZE_VALID)

In [49]:
def model_fit():
    
    
    model = create_model()
    
    loss = tf.keras.losses.CategoricalCrossentropy(from_logits = False,
                                                   label_smoothing=0.001,
                                                   name='categorical_crossentropy')
    
    model.compile(optimizer = Adam(learning_rate = 1e-4),
                        loss = loss,
                        metrics = ['categorical_accuracy']) 
    
    es = EarlyStopping(monitor='val_loss', mode='min', patience=5,
                       restore_best_weights=True, verbose=1)
    
    checkpoint_cb = ModelCheckpoint("test_model_best.h5",
                                    save_best_only=True,
                                    monitor = 'val_loss',
                                    mode='min')
    
    training = model.fit(train_set,
                             validation_data = validation_set,
                             epochs= EPOCHS,
                             batch_size = BATCH_SIZE,
                             steps_per_epoch = STEP_SIZE_TRAIN,
                             validation_steps = STEP_SIZE_VALID,
                             callbacks=[es, checkpoint_cb])
    
    model.save('test_model'+'.h5')  
    
    return training

In [ ]:

train = model_fit()


acc = train.history['categorical_accuracy']
val_acc = train.history['val_categorical_accuracy']

loss = train.history['loss']
val_loss = train.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()